In [4]:
from datetime import datetime, timedelta
import requests
import pandas as pd
import hopsworks
import os
import warnings
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry


## GET THE AIR QUALITY FORECAST FOR THE NEXT DAYS

# Here hopsworks key is not a secret 

with open('../../data/hopsworks-api-key.txt', 'r') as file:
    os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()

## Here it is

# Get the API key from GitHub Secrets


#Get AQI API KEY from secrets of hopsworks

proj = hopsworks.login(project="ID2223LAB1KTH")

conn = hopsworks.connection(host="c.app.hopsworks.ai", project=proj, api_key_value=os.environ.get('HOPSWORKS_API_KEY'))
secrets = conn.get_secrets_api()

AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value


country="sweden"
city = "stockholm"
street = "stockholm-hornsgatan-108"
aqicn_url="https://api.waqi.info/feed/@10009"


# Fetch air quality data from the AQICN API
response = requests.get(f"{aqicn_url}/?token={AQI_API_KEY}")

# Check if the request was successful
if response.status_code == 200:
    data = response.json()

    # 5. Parse the data 
    pm25_data = data.get("data", {}).get("forecast", {}).get("daily", {}).get("pm25", [])
    pm10_data = data.get("data", {}).get("forecast", {}).get("daily", {}).get("pm10", [])


    pm25_df = pd.DataFrame(pm25_data)
    pm10_df = pd.DataFrame(pm10_data)

    pm25_df = pm25_df[['day', 'avg']].rename(columns={'avg': 'pm25'})
    pm10_df = pm10_df[['day', 'avg']].rename(columns={'avg': 'pm10'})
    

    df_forecast_aq = pm25_df.merge(pm10_df, on="day", how="inner").drop(index=0)
    df_forecast_aq[['pm25', 'pm10']] = df_forecast_aq[['pm25', 'pm10']].astype(int)
   
    df_forecast_aq['day'] = pd.to_datetime(df_forecast_aq['day'])
    df_forecast_aq.rename(columns={'day': 'date'}, inplace=True)
    
df_forecast_aq

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1170583
Connected. Call `.close()` to terminate connection gracefully.


,date,pm25,pm10
1,2024-11-13,15,8
2,2024-11-14,9,6
3,2024-11-15,10,8
4,2024-11-16,6,4
5,2024-11-17,4,4
6,2024-11-18,8,7
7,2024-11-19,5,5
